In [ ]:
# This notebook is used to exclude rDNA in KARR-seq and map snoRNA-non-rRNA interactions
# in this pipeline, fastp is used to merge and dedupe paired end data
import os
from tqdm.auto import tqdm, trange
import pysam
import re
import pybedtools
samtools = '/tools/bin/samtools'
import subprocess
import pandas as pd
import numpy as np
from functools import reduce
STAR = '/tools/bin/STAR'
fastp = '/tools/bin/fastp'

In [ ]:
def CIGARdiv(CIGAR):
    #considers all CIGAR operations [MINDSPH=X]
    #divide a cigar string to N-separated segments, used in functions below
    #return these results:
    #gaps, a list of N strings, e.g. ['100N', '200N']
    #segs, a list of segments, each with all possible operations except N
    #gaplens, a list of N lengths, e.g. [100, 200]
    #Glen, genomic length of the entire alignment (consumed ref) [MDN=X]
    #Mlens, segment lengths of matches only [M=X]
    #Rlens, segment lengths of consumed Reference [MD=X]
    #Qlens, segment lengths of consumed Query [MIS=X]
    #example: 
    #Glen,gaps,gaplens,segs,Mlens,Qlens,Rlens = CIGARdiv(CIGAR)
    #or replace unwanted output with '_' 
    gaps=re.findall('\d+N', CIGAR)
    gaplens=[int(gap[:-1]) for gap in gaps] #gap lengths 
    segs=[i.rstrip('0123456789') for i in CIGAR.split('N')]
    Mlens=[sum([int(i[:-1]) for i in re.findall('\d+[M=X]',s)]) for s in segs]
    Rlens=[sum([int(i[:-1]) for i in re.findall('\d+[MD=X]',s)]) for s in segs]
    Qlens=[sum([int(i[:-1]) for i in re.findall('\d+[MIS=X]',s)]) for s in segs]
    Glen=sum(gaplens+Rlens)
    return Glen, gaps, gaplens, segs, Mlens, Qlens, Rlens

def get_bam_len(path):
    '''
    count the length of a bam file given the path
    '''
    cmd = f"{samtools} view -@ 40 -c {path}"
    p = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True)
    bam_len = int(p.communicate()[0])
    return bam_len

In [ ]:
# data directory and sample names
input_prefix = '/home/beiliu/data/220125_snoKARR_seq/'

samplename = ['HepG2_50ASO_ribo+_1','HepG2_50ASO__ribo+_2',
              'HepG2_50ASO__ribo-_1', 'HepG2_50ASO__ribo-_2',
              'HepG2_100ASO__ribo-_1', 'HepG2_100ASO_ribo-_2']

# Pre-process fasta file using fastp

In [ ]:
# a list of input gz file names
input_names = os.listdir(f'{input_prefix}')
input_names = sorted([n for n in input_names if n.endswith('gz')])
if len(input_names)!=2*len(samplename):
    print('!!!!!!warning, double check input_names and samplename')
input_names

In [ ]:
trimed_dir = f'{input_prefix}/fastp_processed'
os.system(f'mkdir {trimed_dir}')
for i in trange(int(len(input_names)/2)):
    # merge and trim adapter
    input1 = f'{input_prefix}/' + input_names[i*2]
    input2 = f'{input_prefix}/' + input_names[i*2+1]
    print(f'inputs are\n 1.{input1}\n 2.{input2}\n')
    
    os.system(f'{fastp} --in1 {input1} \
    --in2 {input2} \
    -m \
    --out1 {trimed_dir}/unmerged_out1.fasta.gz \
    --out2 {trimed_dir}/unmerged_out2.fasta.gz \
    --merged_out {trimed_dir}/merged{i+1}.fasta.gz \
    -w 16 \
    -h {trimed_dir}/merge_and_trim_adapter_{i+1}.html \
    -j {trimed_dir}/merge_and_trim_adapter_{i+1}.json')
    
    # dedupe and trim XXX
    os.system(f'{fastp} -i {trimed_dir}/merged{i+1}.fasta.gz \
    -D -t 3 -w 16 \
    -o {trimed_dir}/merged{i+1}_dedupe.fasta.gz \
    -h {trimed_dir}/dedupe_and_trim_xxx_{i+1}.html \
    -j {trimed_dir}/dedupe_and_trim_xxx_{i+1}.json')
    
os.system(f'rm {trimed_dir}/unmerged*')

# Map to pure rDNA

In [ ]:
gen_dir_rDNA = '/home/beiliu/Database/Ref/human_rDNA_repeat/pure_rDNA/STAR_index'
for i, name_prefix in tqdm(enumerate(samplename), total = len(samplename)):
#     if i ==0:
#         continue
    name = '{:03d}_{}_round1'.format(i+1, name_prefix)
    input1 = f'{input_prefix}/fastp_processed/merged{i+1}_dedupe.fasta.gz'
    print(f'1st round STAR alignment input is {input1}')
    os.system(f'{STAR} --runThreadN 40 \
    --runMode alignReads \
    --genomeDir {gen_dir_rDNA} \
    --readFilesIn {input1}  \
    --outFileNamePrefix {input_prefix}/star_dedupe_alignment_pure_rDNA/{name} \
    --readFilesCommand zcat \
    --outReadsUnmapped Fastx  \
    --outFilterMultimapNmax 10 \
    --outFilterScoreMinOverLread 0 \
    --outFilterMatchNminOverLread 0 \
    --outSAMattributes All \
    --outSAMtype BAM Unsorted \
    --alignIntronMin 1 \
    --scoreGap 0 \
    --scoreGapNoncan 0 \
    --scoreGapGCAG 0 \
    --scoreGapATAC 0 \
    --scoreGenomicLengthLog2scale -1 \
    --chimFilter None \
    --chimOutType WithinBAM HardClip \
    --chimSegmentMin 5 \
    --chimJunctionOverhangMin 5 \
    --chimScoreJunctionNonGTAG 0 \
    --chimScoreDropMax 80 \
    --chimNonchimScoreDropMin 20 \
    --peOverlapNbasesMin 12 \
    --peOverlapMMp 0.05 \
    --limitOutSJcollapsed 10000000')

# First round map to genome

In [ ]:
# map unmapped fasta file to genome
# first round
gen_dir = '/home/beiliu/Database/Ref/STAR_index'
for i, name_prefix in tqdm(enumerate(samplename), total = len(samplename)):
    if i ==0:
        continue
    name = '{:03d}_{}_ex_rDNA_round1'.format(i+1, name_prefix)
    name_0 = '{:03d}_{}_round1'.format(i+1, name_prefix)
    input1 = f'{input_prefix}/star_dedupe_alignment_pure_rDNA/{name_0}Unmapped.out.mate1'
    print(f'1st round STAR alignment input is {input1}')
    os.system(f'{STAR} --runThreadN 40 \
    --runMode alignReads \
    --genomeDir {gen_dir} \
    --readFilesIn {input1}  \
    --outFileNamePrefix {input_prefix}/star_dedupe_alignment_exrDNA_1st/{name} \
    --readFilesCommand cat \
    --outReadsUnmapped Fastx  \
    --outFilterMultimapNmax 20 \
    --chimMultimapNmax 100 \
    --outFilterScoreMinOverLread 0 \
    --outFilterMatchNminOverLread 0 \
    --outSAMattributes All \
    --outSAMtype BAM SortedByCoordinate \
    --alignIntronMin 1 \
    --scoreGap 0 \
    --scoreGapNoncan 0 \
    --scoreGapGCAG 0 \
    --scoreGapATAC 0 \
    --scoreGenomicLengthLog2scale -1 \
    --chimFilter None \
    --chimOutType WithinBAM HardClip \
    --chimSegmentMin 5 \
    --chimJunctionOverhangMin 5 \
    --chimScoreJunctionNonGTAG 0 \
    --chimScoreDropMax 80 \
    --chimNonchimScoreDropMin 20 \
    --peOverlapNbasesMin 12 \
    --peOverlapMMp 0.05 \
    --limitOutSJcollapsed 10000000')

# Softreverse

In [ ]:
minlen = 5 #minimal length of the softclipped shorter segment
os.system('mkdir {}/star_dedupe_alignment_exrDNA_1st'.format(input_prefix))
for i, name_prefix in tqdm(enumerate(samplename), total = len(samplename)):
    if i == 0:
        continue
    # specify input and output bam files
    name = '{:03d}_{}_ex_rDNA_round1'.format(i+1, name_prefix)
    bamfile_path = f'{input_prefix}/star_dedupe_alignment_exrDNA_1st/{name}' +\
    'Aligned.sortedByCoord.out.bam'
    inputbam = pysam.AlignmentFile(f'{bamfile_path}', 'rb')
    outputbam = pysam.AlignmentFile('{}/star_dedupe_alignment_exrDNA_1st/{:03d}_{}_converted.bam'.format(input_prefix, i+1, name_prefix), 
                                    'wb', template = inputbam)

    #get bam file length
    cmd = f"{samtools} view -@ 40 {bamfile_path} | wc -l"
    p = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True)
    bam_len = int(p.communicate()[0])

    # Loop through the bam file and perform the reverse
    count = 0
    for j, line in tqdm(enumerate(inputbam), total=bam_len):
        count+=1
        FLAG = line.flag
        if FLAG>=256 and bin(FLAG)[-9]=='1' or line.has_tag('SA'):
            continue #ignore secondary (FLAG=256) and chiastic alignments (SA:Z).
        CIGAR = line.cigarstring
        subMS = re.findall('\d+[MS]', CIGAR) #substrings for M and S
        softs = re.findall('\d+S', CIGAR)
        softslen = [int(k[:-1]) for k in softs]    
        if softslen and max(softslen) >= minlen:
            QNAME, SEQ, QUAL = line.qname, line.seq, line.qual
            if 'S' in subMS[0] and 'S' not in subMS[-1]:
                SEQn = SEQ[softslen[0]:] + SEQ[:softslen[0]]
                QUALn = QUAL[softslen[0]:] + QUAL[:softslen[0]]
            elif 'S' not in subMS[0] and 'S' in subMS[-1]:
                SEQn = SEQ[-softslen[0]:] + SEQ[:-softslen[0]]
                QUALn = SEQ[-softslen[0]:] + SEQ[:-softslen[0]]
            elif 'S' in subMS[0] and 'S' in subMS[-1]:
                SEQn = SEQ[-softslen[1]:] + \
                      SEQ[softslen[0]:-softslen[1]] + SEQ[:softslen[0]] 
                QUALn = QUAL[-softslen[1]:] + \
                       QUAL[softslen[0]:-softslen[1]] + QUAL[:softslen[0]]  
            line.seq = SEQn
            line.qual = QUALn
            outputbam.write(line)

    inputbam.close()
    outputbam.close()


# convert softreverse bam to fastq

In [ ]:
for i, name_prefix in tqdm(enumerate(samplename), total = len(samplename)):
    # convert converted_bam to converted fastq
    os.system(f'{samtools} bam2fq -@ 40 \
    {input_prefix}/star_dedupe_alignment_exrDNA_1st/{i+1:03d}_{name_prefix}_converted.bam \
    | gzip > {input_prefix}/star_dedupe_alignment_exrDNA_1st/{i+1:03d}_{name_prefix}_converted.fastq.gz')

# 2nd round of mapping to genome

In [ ]:
for i, name_prefix in tqdm(enumerate(samplename), total = len(samplename)):
    name = '{:03d}_{}_exrDNA_round2'.format(i+1, name_prefix)
    input1 = f'{input_prefix}/star_dedupe_alignment_exrDNA_1st/{i+1:03d}_{name_prefix}_converted.fastq.gz'
    print(f'2nd round STAR alignment input is {input1}')
    os.system(f'{STAR} --runThreadN 40 \
    --runMode alignReads \
    --genomeDir {gen_dir} \
    --readFilesIn {input1}  \
    --outFileNamePrefix {input_prefix}/star_dedupe_alignment_exrDNA_2nd/{name} \
    --readFilesCommand zcat \
    --outReadsUnmapped Fastx  \
    --outFilterMultimapNmax 20 \
    --chimMultimapNmax 100 \
    --outFilterScoreMinOverLread 0 \
    --outFilterMatchNminOverLread 0 \
    --outSAMattributes All \
    --outSAMtype BAM SortedByCoordinate \
    --alignIntronMin 1 \
    --scoreGap 0 \
    --scoreGapNoncan 0 \
    --scoreGapGCAG 0 \
    --scoreGapATAC 0 \
    --scoreGenomicLengthLog2scale -1 \
    --chimFilter None \
    --chimOutType WithinBAM HardClip \
    --chimSegmentMin 5 \
    --chimJunctionOverhangMin 5 \
    --chimScoreJunctionNonGTAG 0 \
    --chimScoreDropMax 80 \
    --chimNonchimScoreDropMin 20 \
    --peOverlapNbasesMin 12 \
    --peOverlapMMp 0.05 \
    --limitOutSJcollapsed 10000000 \
    --limitBAMsortRAM 50657646430')

# combine bam files from 2 rounds of mapping

In [ ]:
# directory for combined bam files from 2 rounds of mapping
os.system(f'mkdir -p {input_prefix}/bam_from_2_rounds_exrDNA')
for i, name_prefix in tqdm(enumerate(samplename), total = len(samplename)):
    bam_1 = '{}/star_dedupe_alignment_exrDNA_1st/{:03d}_{}_ex_rDNA_round1'.format(input_prefix, i+1, name_prefix) +\
    'Aligned.sortedByCoord.out.bam'
    bam_2 = '{}/star_dedupe_alignment_exrDNA_2nd/{:03d}_{}_exrDNA_round2'.format(input_prefix, i+1, name_prefix) +\
    'Aligned.sortedByCoord.out.bam'
    os.system(f'{samtools} merge -@ 40 -f \
    {input_prefix}/bam_from_2_rounds_exrDNA/{i+1:03d}_{name_prefix}_exrDNA_total.bam {bam_1} {bam_2}')

# get chimeric bam and process chimeric reads

In [ ]:
hg38_rRNA_bed = pybedtools.BedTool('/home/beiliu/Database/Repeats/UCSC_repeat_masker_rRNA.bed')
snoDB_bed = pybedtools.BedTool('/home/beiliu/Database/Bed_files/snoDB.bed')
repeat_bed = pybedtools.BedTool('/home/beiliu/Database/Repeats/UCSC_repeat_masker.bed')
genome_gtf_path = '/home/beiliu/Database/annotation/hg38/gencode.v39.annotation.gtf'
ref = 'hg38'

In [ ]:
for i, name_prefix in tqdm(enumerate(samplename), total = len(samplename)):
    # if i == 0:
    #     continue
    output_foldername = f'{input_prefix}/bam_from_2_rounds_exrDNA/{i+1:03d}_chimeric_reads_processing'
    os.system(f'mkdir -p {output_foldername}')
    
    bamfile = f'{input_prefix}/bam_from_2_rounds_exrDNA/{i+1:03d}_{name_prefix}_exrDNA_total.bam'
    
    # first, extract chimeric reads from bam file
    cmd1 = f'{samtools} view -@ 40 -Shb -e \"[SA]\" \
    {bamfile}  > {output_foldername}/chimall.bam'
    os.system(cmd1)
    
    # 2nd, extract primary alignment from bam file
    os.system(f'{samtools} view -hub -@ 40 -F 1284 \
    {output_foldername}/chimall.bam > {output_foldername}/chimall_primary.bam')
    
    # 3rd directly overlap chimall.bam with snoDB.bed:  
    os.system(f'bedtools intersect -abam {output_foldername}/chimall_primary.bam -b \
    ~/Database/Bed_files/snoDB.bed -S -u > {output_foldername}/snoDB_overlapped_chimall.bam')
    
    # 4. get unique qname from snoDB_overlapped_chimall.bam
    os.system(f'{samtools} view -@ 40 {output_foldername}/snoDB_overlapped_chimall.bam \
    | cut -f1 | sort | uniq > {output_foldername}/snoDB_qname.txt')
    
    # 5. select reads from chimall.bam that contains qname from snoDB_overlapped_chimall.bam
    os.system(f'{samtools} view -@ 60 -N {output_foldername}/snoDB_qname.txt \
    -o {output_foldername}/snoDB_chimall_filtered_qname.bam {output_foldername}/chimall_primary.bam')
    
    # 6. use pysam to read all alignment and get chr, start, cigar, strand information of both arms and generate bed
    chimall_path = f'{output_foldername}/snoDB_chimall_filtered_qname.bam'
    input_chimall_bam = pysam.AlignmentFile(chimall_path, 'rb')
    chimall_bam_len = get_bam_len(chimall_path)
    
    # 7. generate chimeric reads bed files
    with open(f'{output_foldername}/snoDB_chimall_filtered_qname_sorted_left.bed', 'w') as f1, \
    open (f'{output_foldername}/snoDB_chimall_filtered_qname_sorted_right.bed', 'w') as f2:
        for i, line in tqdm(enumerate(input_chimall_bam), total=chimall_bam_len):
            ID = i+1
            qname = line.qname
            chr1, pos1, strand1, cigar1 = \
                    line.reference_name, line.pos+1, "-" if line.is_reverse else '+', line.cigarstring
            chr2, pos2, strand2, cigar2, _, _ = [i for i in line.tags if i[0]=='SA'][0][1].split(',')
            pos2 = int(pos2)

            end1 = pos1+CIGARdiv(cigar1)[0]-1
            end2 = int(pos2)+CIGARdiv(cigar2)[0]-1

            # get rid of intra-molecular interactions
            if chr1 == chr2:
                overlap = max(0, min(end1, end2) - max(pos1, pos2))
                if overlap != 0:
                    continue

            # correct the start position for bed file output
            line1 = [chr1, str(pos1-1), str(end1), str(ID), qname, strand1]
            line1 = '\t'.join(line1)
            # print(line)
            f1.write(f"{line1}\n")

            # correct the start position for bed file output
            line2 = [chr2, str(pos2-1), str(end2), str(ID), qname, strand2]
            line2 = '\t'.join(line2)
            # print(line)
            f2.write(f"{line2}\n")
            
    # 8. process chimeric bed files
    # next overlap left and right arm with rRNA to exclude rRNA
    left_bed = pybedtools.BedTool(f'{output_foldername}/snoDB_chimall_filtered_qname_sorted_left.bed')
    right_bed = pybedtools.BedTool(f'{output_foldername}/snoDB_chimall_filtered_qname_sorted_right.bed')

    # remove rRNA
    left_norRNA = left_bed.intersect(hg38_rRNA_bed, v = True, wa = True, S = True).to_dataframe()
    right_norRNA = right_bed.intersect(hg38_rRNA_bed, v = True, wa = True, S = True).to_dataframe()       
    
    
    # merge left and right that have the same ID (neither arm overlaps with rRNA)
    norRNA_chimeric_df = pd.merge(left_norRNA, right_norRNA, on = 'name', how = 'inner')

    # create full nema columns to sort each row
    norRNA_chimeric_df['arm1'] = norRNA_chimeric_df['chrom_x'] + '$' +\
    norRNA_chimeric_df['start_x'].astype(str) + '$' +\
    norRNA_chimeric_df['end_x'].astype(str) + '$' +\
    norRNA_chimeric_df['strand_x'] + '$' + \
    norRNA_chimeric_df['name'].astype(str) + '$' +\
    norRNA_chimeric_df['score_x']

    norRNA_chimeric_df['arm2'] = norRNA_chimeric_df['chrom_y'] + '$' +\
    norRNA_chimeric_df['start_y'].astype(str) + '$' +\
    norRNA_chimeric_df['end_y'].astype(str) + '$' +\
    norRNA_chimeric_df['strand_y'] + '$' + \
    norRNA_chimeric_df['name'].astype(str) + '$' +\
    norRNA_chimeric_df['score_y']
    
    # sort each row
    df = norRNA_chimeric_df[['arm1', 'arm2']]
    sort_row_chim = pd.DataFrame(np.sort(df.values, axis=1)[:,::-1], 
                 index=df.index, 
                 columns=df.columns)
    # recover coordinates
    arm1_df = sort_row_chim['arm1'].str.split('$', expand = True)
    arm2_df = sort_row_chim['arm2'].str.split('$', expand = True)

    # concat two arms
    arm1_arm2_df = pd.concat([arm1_df, arm2_df], axis=1)

    arm1_arm2_df.columns = ['chr1', 'start1', 'end1', 'strand1', 'ID', 'qname', 
                           'chr2', 'start2', 'end2', 'strand2', 'ID2', 'qname2']

    arm1_arm2_df = arm1_arm2_df[['chr1', 'start1', 'end1', 'strand1', 'ID', 'qname', 
                           'chr2', 'start2', 'end2', 'strand2']]

    # drop qname duplicates, each chimeric reads have two entries with the same qname
    arm1_arm2_df = arm1_arm2_df.drop_duplicates(subset = 'qname').reset_index(drop=True)
    
    # now make arm1 snoRNA and arm2 target
    arm1_bed_df = arm1_arm2_df[['chr1','start1','end1','ID','qname','strand1']]
    arm2_bed_df = arm1_arm2_df[['chr2','start2','end2','ID','qname','strand2']]

    arm1_bed = pybedtools.BedTool.from_dataframe(arm1_bed_df)
    arm2_bed = pybedtools.BedTool.from_dataframe(arm2_bed_df)

    # intersect left and right arms with snoDB
    arm1_bed_snoRNA = arm1_bed.intersect(snoDB_bed, wa = True, wb = True, u = True, S = True)
    arm2_bed_snoRNA = arm2_bed.intersect(snoDB_bed, wa = True, wb = True, u = True, S = True)

    arm1_bed_snoRNA_df = arm1_bed_snoRNA.to_dataframe()
    arm2_bed_snoRNA_df = arm2_bed_snoRNA.to_dataframe()

    # left arm is snoRNA
    left_arm_snoRNA = arm1_arm2_df[arm1_arm2_df['qname'].isin(arm1_bed_snoRNA_df['score'])]
    # right arm is snoRNA, switch chr1 and chr2 coordinates
    right_arm_snoRNA = arm1_arm2_df[arm1_arm2_df['qname'].isin(arm2_bed_snoRNA_df['score'])]
    right_arm_snoRNA = right_arm_snoRNA[['chr2','start2','end2','strand2', 'ID','qname', 'chr1','start1','end1','strand1']]
    right_arm_snoRNA.columns = left_arm_snoRNA.columns

    # drop duplicates that both arms are snoRNAs
    reordered_snoRNA_target_df = pd.concat([left_arm_snoRNA, right_arm_snoRNA]).drop_duplicates(subset = 'qname')
    
    # now split reordered_snoRNA_target_df to left and right bed files, run annotation
    snoRNA_bed_df = reordered_snoRNA_target_df[['chr1','start1','end1','qname','ID','strand1']]
    target_bed_df = reordered_snoRNA_target_df[['chr2','start2','end2','qname','ID','strand2']]

    pybedtools.BedTool.from_dataframe(snoRNA_bed_df).saveas(f'{output_foldername}/snoRNA.bed')
    pybedtools.BedTool.from_dataframe(target_bed_df).saveas(f'{output_foldername}/target.bed')

    # remove intermediate files
    os.system(f'rm {output_foldername}/*.bam')
    os.system(f'rm {output_foldername}/snoDB_qname.txt')
    os.system(f'rm {output_foldername}/snoDB_chimall_filtered_qname_sorted_*.bed')

    # Run annotation for both arms
    with open (f'{output_foldername}/annotate1.sh', 'w') as hd:
        hd.write(f'#!/bin/bash\nexport PATH=/home/beiliu/anaconda3/bin/:$PATH\nannotatePeaks.pl \
        {output_foldername}/snoRNA.bed {ref} -gtf {genome_gtf_path}\
        -annStats {output_foldername}/snoRNA_stats_annote.txt -cpu 40 > {output_foldername}/snoRNA_annotate.txt')
    os.system(f'nohup bash {output_foldername}/annotate1.sh &')

    with open (f'{output_foldername}/annotate2.sh', 'w') as hd:
        hd.write(f'#!/bin/bash\nexport PATH=/home/beiliu/anaconda3/bin/:$PATH\nannotatePeaks.pl \
        {output_foldername}/target.bed {ref} -gtf {genome_gtf_path}\
        -annStats {output_foldername}/target_stats_annote.txt -cpu 40 > {output_foldername}/target_annotate.txt')
    os.system(f'nohup bash {output_foldername}/annotate2.sh &')

# process annotation files

In [ ]:
for i, name_prefix in tqdm(enumerate(samplename), total = len(samplename)):
    data_dir = f'{input_prefix}/bam_from_2_rounds_exrDNA/{i+1:03d}_chimeric_reads_processing'
    
    # read in annotation files
    anote_snoRNA = pd.read_csv(f'{data_dir}/snoRNA_annotate.txt', delimiter='\t')
    anote_snoRNA = anote_snoRNA.rename(columns={anote_snoRNA.columns[0]:'ID'})
    anote_snoRNA = anote_snoRNA[['Chr', 'Start', 'End', 'Strand', 'Annotation', 'Detailed Annotation', 'Distance to TSS', 'Gene Name', 'Gene Type', 'ID']]

    anote_target = pd.read_csv(f'{data_dir}/target_annotate.txt', delimiter='\t')
    anote_target = anote_target.rename(columns={anote_target.columns[0]:'ID'})
    anote_target = anote_target[['Chr', 'Start', 'End', 'Strand', 'Annotation', 'Detailed Annotation', 'Distance to TSS', 'Gene Name', 'Gene Type', 'ID']]
    
    # directly annotate snoRNA using snoDB
    snoRNA_total_bed = pybedtools.BedTool.from_dataframe(anote_snoRNA[['Chr', 'Start', 'End', 'ID', 'Gene Name', 'Strand']])
    totalsnoRNA_snoDB_annot = snoRNA_total_bed.intersect(snoDB_bed, wa = True, wb = True, S = True).to_dataframe()
    snoDB_snoRNA_annotation = totalsnoRNA_snoDB_annot.drop_duplicates('name')[['name', 'blockCount']]
    snoDB_snoRNA_annotation.columns = ['ID', 'snoRNA_snoDB_annotation']

    # directly annotate snoRNA using repeat bed
    totalsnoRNA_snoDB_annot_repeat = snoRNA_total_bed.intersect(repeat_bed, wa = True, wb = True, S = True).to_dataframe()
    totalsnoRNA_snoDB_annot_repeat = totalsnoRNA_snoDB_annot_repeat.drop_duplicates('name')[['name', 'blockCount']]
    totalsnoRNA_snoDB_annot_repeat.columns = ['ID', 'snoRNA_repeat_annotation']

    # merge annotation and snoDB annotation
    snoRNA_target_annotation_df = reduce(lambda left, right: pd.merge(left, right, on = 'ID', how = 'inner'), 
                                         [anote_snoRNA, anote_target, snoDB_snoRNA_annotation])
    snoRNA_target_annotation_df = snoRNA_target_annotation_df.dropna()
    snoRNA_target_annotation_df = snoRNA_target_annotation_df[~snoRNA_target_annotation_df['Gene Type_y'].str.contains('rRNA|snoRNA')]
    snoRNA_target_annotation_df.to_csv(f'{data_dir}/snoRNA_target_annotation.csv', index = False)
    
    # merge annotation total and snoRNA annotated from repeat
    repeat_annotated_snoRNA_df = pd.merge(snoRNA_target_annotation_df, totalsnoRNA_snoDB_annot_repeat, on = 'ID', how = 'inner')
    repeat_annotated_snoRNA_df.to_csv(f'{data_dir}/snoRNA_target_annotation_snoRNA_annotated_by_repeat.csv', index = False)

In [ ]:
# read in all annotation files
total_annotation_lst = []
snoRNA_annotatedbyrepeats_annotation_lst = []
for i, name_prefix in tqdm(enumerate(samplename), total = len(samplename)):
    # if i == 0:
    #     continue
    data_dir = f'{input_prefix}/bam_from_2_rounds_exrDNA/{i+1:03d}_chimeric_reads_processing'
    
    total_annotation = pd.read_csv(f'{data_dir}/snoRNA_target_annotation.csv')
    snoRNA_annotatedbyrepeats_annotation = pd.read_csv(f'{data_dir}/snoRNA_target_annotation_snoRNA_annotated_by_repeat.csv')
    
    total_annotation_lst.append(total_annotation)
    snoRNA_annotatedbyrepeats_annotation_lst.append(snoRNA_annotatedbyrepeats_annotation)

In [ ]:
# get the unique chromosome of snoRNA and targets for the first two data
data1 = total_annotation_lst[0]
data2 = total_annotation_lst[1]

# the idea is that for each chr and strand in left arm and right arm, run DG clustering
snoRNA_common_chr = np.intersect1d(data1['Chr_x'].unique(), data2['Chr_x'].unique())
target_common_chr = np.intersect1d(data1['Chr_y'].unique(), data2['Chr_y'].unique())

# Run DG clustering

In [ ]:
%run ~/KARR_seq/Process_function_notebooks/DG_sim.ipynb

In [ ]:
DG_dir = f'{input_prefix}/bam_from_2_rounds_exrDNA/DG_clustering_001_002'
os.system(f'mkdir -p {DG_dir}')

# for the first two replicate data
# loop through the snoRNA and target chromosome, run DG clustering
for i, chr1 in tqdm(enumerate(snoRNA_common_chr), total = len(snoRNA_common_chr)):

    print(f'processing snoRNA chromosome {chr1}')
    # split + and - dataset
    chr1_pos_data1 = data1[(data1['Chr_x']==chr1) & (data1['Strand_x']=='+')]
    chr1_pos_data2 = data2[(data2['Chr_x']==chr1) & (data2['Strand_x']=='+')]
    chr1_neg_data1 = data1[(data1['Chr_x']==chr1) & (data1['Strand_x']=='-')]
    chr1_neg_data2 = data2[(data2['Chr_x']==chr1) & (data2['Strand_x']=='-')]
    for j, chr2 in tqdm(enumerate(target_common_chr), total = len(target_common_chr)):

        print(f'processing target chromosome {chr2}')
        chr1_pos_chr2_pos_data1 = chr1_pos_data1[(chr1_pos_data1['Chr_y']==chr2) & (chr1_pos_data1['Strand_y']=='+')]
        chr1_pos_chr2_pos_data2 = chr1_pos_data2[(chr1_pos_data2['Chr_y']==chr2) & (chr1_pos_data2['Strand_y']=='+')]
        
        chr1_pos_chr2_neg_data1 = chr1_pos_data1[(chr1_pos_data1['Chr_y']==chr2) & (chr1_pos_data1['Strand_y']=='-')]
        chr1_pos_chr2_neg_data2 = chr1_pos_data2[(chr1_pos_data2['Chr_y']==chr2) & (chr1_pos_data2['Strand_y']=='-')]
        
        chr1_neg_chr2_pos_data1 = chr1_neg_data1[(chr1_neg_data1['Chr_y']==chr2) & (chr1_neg_data1['Strand_y']=='+')]
        chr1_neg_chr2_pos_data2 = chr1_neg_data2[(chr1_neg_data2['Chr_y']==chr2) & (chr1_neg_data2['Strand_y']=='+')]
        
        chr1_neg_chr2_neg_data1 = chr1_neg_data1[(chr1_neg_data1['Chr_y']==chr2) & (chr1_neg_data1['Strand_y']=='-')]
        chr1_neg_chr2_neg_data2 = chr1_neg_data2[(chr1_neg_data2['Chr_y']==chr2) & (chr1_neg_data2['Strand_y']=='-')]
        
        # run clustering for each variation
        for m,n in zip([chr1_pos_chr2_pos_data1, chr1_pos_chr2_neg_data1, chr1_neg_chr2_pos_data1, chr1_neg_chr2_neg_data1], 
                      [('+', '+'), ('+', '-'), ('-', '+'), ('-', '-')]):
            data_filtred = m[['Chr_x','Start_x','End_x','Chr_y','Start_y','End_y']].reset_index(drop=True)
            data_filtred.columns = ['gene1', 'start1', 'end1', 'gene2', 'start2', 'end2']
            run_total_clustering_nonsnoRNA(data_filtred, 0.4, f'{chr1}_{n[0]}', f'{chr2}_{n[1]}', csv_name = f'{DG_dir}/{chr1}_{n[0]}_{chr2}_{n[1]}_rep1', threshold = 1)
            
        for m,n in zip([chr1_pos_chr2_pos_data2, chr1_pos_chr2_neg_data2, chr1_neg_chr2_pos_data2, chr1_neg_chr2_neg_data2], 
                      [('+', '+'), ('+', '-'), ('-', '+'), ('-', '-')]):
            data_filtred = m[['Chr_x','Start_x','End_x','Chr_y','Start_y','End_y']].reset_index(drop=True)
            data_filtred.columns = ['gene1', 'start1', 'end1', 'gene2', 'start2', 'end2']
            run_total_clustering_nonsnoRNA(data_filtred, 0.4, f'{chr1}_{n[0]}', f'{chr2}_{n[1]}', csv_name = f'{DG_dir}/{chr1}_{n[0]}_{chr2}_{n[1]}_rep2', threshold = 1)
            